In [ ]:
# Copyright 2022 Intrinsic Innovation LLC.
# Copyright 2023 DeepMind Technologies Limited.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[Restricted Boltzmann Machine (RBM)](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine) is a well-known and widely used PGM for learning probabilistic distributions over binary data. We demonstrate how we can easily implement [perturb-and-max-product (PMP)](https://proceedings.neurips.cc/paper/2021/hash/07b1c04a30f798b5506c1ec5acfb9031-Abstract.html) sampling from an RBM trained on MNIST digits using PGMax. PMP is a recently proposed method for approximately sampling from a PGM by computing the maximum-a-posteriori (MAP) configuration (using max-product LBP) of a perturbed version of the model.

We start by making some necessary imports.

In [ ]:
# # Uncomment this block if running on colab.research.google.com
# !pip install git+https://github.com/deepmind/PGMax.git
# !wget https://raw.githubusercontent.com/deepmind/PGMax/main/examples/example_data/rbm_mnist.npz
# !mkdir example_data
# !mv rbm_mnist.npz  example_data/

In [ ]:
%matplotlib inline
import functools

import jax
import matplotlib.pyplot as plt
import numpy as np

############
# Load PGMax
from pgmax import fgraph, fgroup, infer, vgroup

The [`pgmax.fgraph`](https://pgmax.readthedocs.io/en/latest/api.html#module-pgmax.fgraph) module contains classes for specifying factor graphs, the [`pgmax.vgroup`](https://pgmax.readthedocs.io/en/latest/api.html#module-pgmax.vgroup) module contains classes for specifying groups of variables, the [`pgmax.fgroup`](https://pgmax.readthedocs.io/en/latest/api.html#module-pgmax.fgroup) module contains classes for specifying groups of factors and the [`pgmax.infer`](https://pgmax.readthedocs.io/en/latest/api.html#module-pgmax.infer) module contains functions to perform LBP.

Next, we load the RBM trained in Sec. 5.5 of the [PMP paper](https://proceedings.neurips.cc/paper/2021/hash/07b1c04a30f798b5506c1ec5acfb9031-Abstract.html) which has been trained on MNIST digits.

In [ ]:
# Load data
folder_name = "example_data/"
params = np.load(open(folder_name + "rbm_mnist.npz", 'rb'), allow_pickle=True)
bv = params["bv"]
bh = params["bh"]
W = params["W"]

We now build the RBM factor graph using PGMax.

First, we use [`NDVarArray`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.vgroup.NDVarArray), a convenient class for specifying a group of variables living on a multidimensional grid with possibly different number of states. This class shares some similarities with [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html), in particular regarding variables indexing. 

Second, we initialize the [`FactorGraph`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.fgraph.FactorGraph) `fg` with the set of defined [`VarGroup`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.vgroup.VarGroup)s. Once initialized, the set of variables in `fg` is fixed and cannot be changed. 

In [ ]:
# Initialize factor graph
hidden_variables = vgroup.NDVarArray(num_states=2, shape=bh.shape)
visible_variables = vgroup.NDVarArray(num_states=2, shape=bv.shape)
fg = fgraph.FactorGraph(variable_groups=[hidden_variables, visible_variables])

After initialization, `fg` does not have any factors. PGMax implements convenient and computationally efficient [`FactorGroup`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.fgroup.FactorGroup) for representing groups of similar [`Factor`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.factor.Factor)s. We then create the unary and pairwise factors using the [`EnumFactorGroup`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.fgroup.EnumFactorGroup) and [`PairwiseFactorGroup`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.fgroup.PairwiseFactorGroup) classes, before adding them to `fg`.

Note that a [`FactorGroup`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.fgroup.FactorGroup) takes as argument `variables_for_factors` which is a list of lists of the variables involved in the different factors, and additional specific arguments (e.g. `factor_configs` and `log_potential_matrix` here).

In [ ]:
# Create unary factors
hidden_unaries = fgroup.EnumFactorGroup(
    variables_for_factors=[[hidden_variables[ii]] for ii in range(bh.shape[0])],
    factor_configs=np.arange(2)[:, None],
    log_potentials=np.stack([np.zeros_like(bh), bh], axis=1),
)
visible_unaries = fgroup.EnumFactorGroup(
    variables_for_factors=[[visible_variables[jj]] for jj in range(bv.shape[0])],
    factor_configs=np.arange(2)[:, None],
    log_potentials=np.stack([np.zeros_like(bv), bv], axis=1),
)

# Create pairwise factors
log_potential_matrix = np.zeros(W.shape + (2, 2)).reshape((-1, 2, 2))
log_potential_matrix[:, 1, 1] = W.ravel()

variables_for_factors = [
    [hidden_variables[ii], visible_variables[jj]]
    for ii in range(bh.shape[0])
    for jj in range(bv.shape[0])
]
pairwise_factors = fgroup.PairwiseFactorGroup(
    variables_for_factors=variables_for_factors,
    log_potential_matrix=log_potential_matrix,
)

# Add factors to the FactorGraph
fg.add_factors([hidden_unaries, visible_unaries, pairwise_factors])

An alternative way displayed below adds the above [`Factor`](https://pgmax.readthedocs.io/en/latest/api.html#pgmax.factor.Factor)s iteratively without building the FactorGroup. This approach is not recommended as it can be much slower than using [`FactorGroup`](https://pgmax.readthedocs.io/en/latest/_autosummary/pgmax.fgroup.fgroup.FactorGroup.html#pgmax.fgroup.fgroup.FactorGroup)s.

~~~python
from pgmax import factor
import itertools
from tqdm import tqdm

# Add unary factors
for ii in range(bh.shape[0]):
  unary_factor = factor.EnumFactor(
      variables=[hidden_variables[ii]],
      factor_configs=np.arange(2)[:, None],
      log_potentials=np.array([0, bh[ii]]),
  )
  fg.add_factors(unary_factor)

for jj in range(bv.shape[0]):
  unary_factor = factor.EnumFactor(
      variables=[visible_variables[jj]],
      factor_configs=np.arange(2)[:, None],
      log_potentials=np.array([0, bv[jj]]),
  )
  fg.add_factors(unary_factor)

# Add pairwise factors
factor_configs = np.array(list(itertools.product(np.arange(2), repeat=2)))
for ii in tqdm(range(bh.shape[0])):
  for jj in range(bv.shape[0]):
    pairwise_factor = factor.EnumFactor(
        variables=[hidden_variables[ii], visible_variables[jj]],
        factor_configs=factor_configs,
        log_potentials=np.array([0, 0, 0, W[ii, jj]]),
    )
    fg.add_factors(pairwise_factor)
~~~

Once we have added the factors, we can run max-product LBP and get the MAP decoding by
~~~python
bp = infer.build_inferer(fg.bp_state, backend="bp")
bp_arrays = bp.run(bp.init(), num_iters=100, damping=0.5, temperature=0.0)
beliefs = bp.get_beliefs(bp_arrays)
map_states = infer.decode_map_states(beliefs)
~~~
or run sum-product LBP and get the estimated marginals by
~~~python
bp = infer.build_inferer(fg.bp_state, backend="bp")
bp_arrays = bp.run(bp.init(), num_iters=100, damping=0.5, temperature=1.0)
beliefs = bp.get_beliefs(bp_arrays)
marginals = infer.get_marginals(beliefs)
~~~
More generally, PGMax implements LBP for any temperature, with `temperature=0.0` and `temperature=1.0` corresponding to the commonly used max/sum-product LBP respectively.

Now we are ready to demonstrate PMP sampling from RBM. PMP perturbs the model with [Gumbel](https://numpy.org/doc/stable/reference/random/generated/numpy.random.gumbel.html) unary potentials, and draws a sample from the RBM as the MAP decoding from running max-product LBP on the perturbed model

In [ ]:
bp = infer.build_inferer(fg.bp_state, backend="bp")

In [ ]:
bp_arrays = bp.init(
    evidence_updates={
        hidden_variables: np.random.gumbel(size=(bh.shape[0], 2)),
        visible_variables: np.random.gumbel(size=(bv.shape[0], 2)),
    }
)
bp_arrays, msgs_deltas = bp.run_with_diffs(
    bp_arrays, num_iters=500, damping=0.5, temperature=0.0
)
beliefs = bp.get_beliefs(bp_arrays)

Here we use the `evidence_updates` argument of `bp.init` to perturb the model with Gumbel unary potentials. In general, `evidence_updates` can be used to incorporate evidence in the form of externally applied unary potentials in PGM inference.

Visualizing the MAP decoding, we see that we have sampled an MNIST digit!

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
map_states = infer.decode_map_states(beliefs)
ax.imshow(
    map_states[visible_variables].copy().reshape((28, 28)),
    cmap="gray",
)
ax.axis("off")

`bp.run_with_diffs` returns the maximum absolute difference of the messages between two consecutive iterations, which is used to monitor the (unguaranteed) convergence of BP

If we do not want to access `msgs_deltas` we can also call 

`bp_arrays = bp.run(bp_arrays, num_iters=500, damping=0.5, temperature=0.0)`

In [ ]:
print(msgs_deltas[-10:])

plt.figure(figsize=(8, 5))
plt.plot(msgs_deltas)
plt.title("Max-product convergence", fontsize=18)
plt.xlabel("BP iteration", fontsize=16)
plt.ylabel("Max abs msgs diff", fontsize=16)

PGMax can compute the energy of a decoding, expressed by its MAP states.

`debug_mode` gives access to the individual contribution of each variable and factor to the energy.

~~~python
decoding_energy, vars_energies, factors_energies = infer.compute_energy(
    fg.bp_state, bp_arrays, map_states, debug_mode=True
)
~~~

In [ ]:
decoding_energy = infer.compute_energy(fg.bp_state, bp_arrays, map_states)[0]
print("The energy of the decoding is", decoding_energy)


PGMax adopts a functional interface for implementing LBP: running LBP in PGMax starts with
~~~python
bp = infer.build_inferer(fg.bp_state, backend="bp")
~~~
where the arguments of `bp` are several useful functions to run LBP. In particular, `bp.init`, `bp.run`, `bp.get_beliefs` are pure functions with no side-effects. This design choice means that we can easily apply JAX transformations like `jit`/`vmap`/`grad`, etc., to these functions, and additionally allows PGMax to seamlessly interact with other packages in the rapidly growing JAX ecosystem (see [here](https://deepmind.com/blog/article/using-jax-to-accelerate-our-research) and [here](https://github.com/n2cholas/awesome-jax)).

As an example of applying `jax.vmap` to `bp.init`/`bp.run`/`bp.get_beliefs` to process a batch of samples/models in parallel, instead of drawing one sample at a time as above, we can draw a batch of samples in parallel as follows:

In [ ]:
n_samples = 10
bp_arrays = jax.vmap(bp.init, in_axes=0, out_axes=0)(
    evidence_updates={
        hidden_variables: np.random.gumbel(size=(n_samples, bh.shape[0], 2)),
        visible_variables: np.random.gumbel(size=(n_samples, bv.shape[0], 2)),
    },
)
bp_arrays = jax.vmap(
    functools.partial(bp.run, num_iters=100, damping=0.5, temperature=0.0),
    in_axes=0,
    out_axes=0,
)(bp_arrays)

beliefs = jax.vmap(bp.get_beliefs, in_axes=0, out_axes=0)(bp_arrays)
map_states = infer.decode_map_states(beliefs)

Visualizing the MAP decodings, we see that we have sampled 10 MNIST digits in parallel!

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(20, 8))
for ii in range(10):
    ax[np.unravel_index(ii, (2, 5))].imshow(
        map_states[visible_variables][ii].copy().reshape((28, 28)), cmap="gray"
    )
    ax[np.unravel_index(ii, (2, 5))].axis("off")

fig.tight_layout()